In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from ollama import Client

client = Client(host="http://localhost:11434")
model_name = "qwen2.5:7b"

In [22]:
from agentic.tool import ToolAgent
from functions import summation, multiplication

In [23]:
agent = ToolAgent(
    client=client, toolkit=[summation, multiplication], model="qwen2.5:7b"
)

In [24]:
response = agent.start(
    message="Can you multiple 25 by 12 and sum 3 with 6 and search what is an LLM on google.com?"
)

In [25]:
from agentic.planning import PlanningAgent

In [26]:
pagent = PlanningAgent(
    client=client, toolkit=[summation, multiplication], model="qwen2.5:7b"
)

In [27]:
result = pagent.start(
    message="Can you multiple 25 by 12 and sum the result by 3 and multiply by 12 and then sum by 354?"
)

In [28]:
print(result)

The final result of the arithmetic operations is 3990. 

<response>The final result after performing all the operations is 3990.</response>


In [31]:
from tool import Function, Tool, ToolKit
from typing import List

In [32]:
def sum2(a: float, b: float) -> float:
    return a + b


def sub(a: float, b: float) -> float:
    return a - b


def list_mul(a: int, b: List = [1, 2, 3]) -> List:
    return a * b

In [33]:
sum2_func = Function(sum2)
sub_func = Function(sub)

In [34]:
sub_func

Function(name=sub, signature=(a: float, b: float) -> float, doc=None)

In [35]:
sum2_func.run({"a": 2, "b": 3})

5.0

In [36]:
sub_func.run({"a": 2, "b": 3})

-1.0

In [37]:
sum2_tool = Tool(func=sum2)

In [38]:
sum2_tool.schema

{'name': 'sum2',
 'doc': None,
 'parameters': {'properties': {'a': {'type': 'float'},
   'b': {'type': 'float'}}}}

In [39]:
sum2_tool.run({"a": 2, "b": 3})

5.0

In [40]:
assert sum2_tool.run({"a": 2, "b": 3}) == sum2_func.run({"a": 2, "b": 3})

In [41]:
sum2_tool.run({"a": 2, "b": 3})

5.0

In [42]:
tk = ToolKit(tools=[sum2, sub, list_mul])

In [43]:
tk.tools

[Tool(name=sum2, signature=(a: float, b: float) -> float, doc=None),
 Tool(name=sub, signature=(a: float, b: float) -> float, doc=None),
 Tool(name=list_mul, signature=(a: int, b: List = [1, 2, 3]) -> List, doc=None)]

In [44]:
assert (
    tk.tools[0].run({"a": 2, "b": 3})
    == sum2_tool.run({"a": 2, "b": 3})
    == sum2_func.run({"a": 2, "b": 3})
)

In [45]:
assert (
    tk.get_tool_by_name("sum2").run({"a": 2, "b": 3})
    == sum2_tool.run({"a": 2, "b": 3})
    == sum2_func.run({"a": 2, "b": 3})
)

In [48]:
from tool import ToolCallProcessor

# Example usage
processor = ToolCallProcessor(
    """
<tool_call>
{"name": "summation", "arguments": {"x": 3, "y": 6}, "id": 2}
</tool_call>
<tool_call>
{"name": "search", "arguments": {"query": "what is an LLM on google.com"}, "id": 3}
</tool_call>
"""
)

In [49]:
for call in processor.calls:
    print(call)